In [25]:
import random

# Define constants
NUM_DAYS = 5
NUM_ROOMS = 4
NUM_SESSIONS = 3
NUM_COURSES = 25
OPTIMAL_COURSES = 6

# Course data
matakuliah = {
    "MII225201": ("Analisis Algoritme", "Selasa", "10:30-13:00"),
    "MII225202": ("Matematika untuk Ilmu Komputer", "Kamis", "13:30-16:00"),
    "MII226001": ("Metodologi Riset Ilmu Komputer", "Jumat", "13:00-15:30"),
    "MII226205": ("Pengolahan dan Analisis Citra Digital", "Kamis", "13:30-16:00"),
    "MII226502": ("Sistem Temu Balik Informasi", "Kamis", "07:30-10:00"),
    "MII226503": ("Data Warehouse dan Inteligensi Bisnis", "Selasa", "07:30-10:00"),
    "MII226504": ("Pengembangan Perangkat Lunak", "Rabu", "13:55-16:25"),
    "MII226505": ("Kecerdasan Digital dan Informatika Sosial", "Senin", "13:30-16:00"),
    "MII226506": ("Manajemen dan Audit Sistem Informasi", "Rabu", "13:30-16:00"),
    "MII226203": ("Teori Komputasi", "Kamis", "10:30-13:00"),
    "MII226401": ("Kecerdasan Komputasional dan Pembelajaran Mesin", "Selasa", "13:30-16:00"),
    "MII226402": ("Prinsip Kecerdasan Artifisial", "Rabu", "07:30-10:00"),
    "MII226403": ("Rekayasa Fitur dan Pengenalan Pola", "Selasa", "07:30-10:00"),
    "MII226404": ("Sistem Pendukung Pembuatan Keputusan", "Kamis", "10:30-13:00"),
    "MII226501": ("Data Science", "Rabu", "10:30-13:00"),
    "MII226601": ("Komputasi Awan dan Keamanan Siber", "Selasa", "13:30-16:00"),
    "MII226602": ("Jaringan Komputer Lanjut", "Rabu", "10:30-13:00"),
    "MII226603": ("Platform dan Arsitektur Big Data", "Selasa", "10:30-13:00"),
}

# Define mappings
hari_kode = {"Senin": 1, "Selasa": 2, "Rabu": 3, "Kamis": 4, "Jumat": 5}
ruangan_kode = {"419": 1, "420": 2, "421": 3, "422": 4}

# Define course penalty dictionary
course_penalties = {course_id: 0 for course_id in matakuliah.keys()}

# Generate initial population
def generate_population(population_size):
    population = []
    for _ in range(population_size):
        chromosome = [random.choice(list(matakuliah.keys())) for _ in range(NUM_DAYS * NUM_ROOMS * NUM_SESSIONS)]
        population.append(chromosome)
    return population

# Calculate fitness for each chromosome
def calculate_fitness(chromosome):
    penalty = 0
    # Calculate penalty for overlapping courses
    schedule = [[set() for _ in range(NUM_SESSIONS)] for _ in range(NUM_DAYS)]
    for i, gene in enumerate(chromosome):
        day = i // (NUM_ROOMS * NUM_SESSIONS)
        room_session = i % (NUM_ROOMS * NUM_SESSIONS)
        room = room_session // NUM_SESSIONS
        session = room_session % NUM_SESSIONS
        if gene in schedule[day][session]:
            penalty += course_penalties[gene]
        else:
            schedule[day][session].add(gene)
    fitness = 1 / (1 + penalty)
    return fitness

# Select parents using roulette wheel selection
def select_parents(population, fitness_values):
    total_fitness = sum(fitness_values)
    selection_probabilities = [fitness / total_fitness for fitness in fitness_values]
    selected_parents = random.choices(population, weights=selection_probabilities, k=2)
    return selected_parents

# Perform one-point crossover
def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# Perform mutation
def mutate(chromosome):
    mutated_chromosome = chromosome.copy()
    index = random.randint(0, len(chromosome) - 1)
    mutated_chromosome[index] = random.choice(list(matakuliah.keys()))
    return mutated_chromosome

# Main genetic algorithm function
def genetic_algorithm(population_size, generations):
    population = generate_population(population_size)
    for _ in range(generations):
        # Calculate fitness for each chromosome
        fitness_values = [calculate_fitness(chromosome) for chromosome in population]

        # Select parents
        parents = select_parents(population, fitness_values)

        # Perform crossover
        # Perform crossover
        offspring = []
        for i in range(0, len(parents) - 1, 2):
            child1, child2 = crossover(parents[i], parents[i+1])
            offspring.append(child1)
            offspring.append(child2)

        
        # If population size is odd, add one random parent to the next generation
        if population_size % 2 == 1:
            offspring.append(parents[-1])

        # Perform mutation
        mutated_offspring = [mutate(child) for child in offspring]

        # Replace old population with new population
        population = mutated_offspring

        # Sort population by fitness
        population.sort(key=lambda x: calculate_fitness(x), reverse=True)

        # Select top individuals
        population = population[:population_size]

        # Early stopping if we find the optimal solution
        if max(fitness_values) == 1:
            break

    return population

# Main function
def main():
    population_size = 10
    generations = 1000
    optimal_solution = genetic_algorithm(population_size, generations)
    print("Optimal Schedule:")
    for course_id in optimal_solution[1][:OPTIMAL_COURSES]:
        print("Course:", course_id, "-", matakuliah[course_id])

if __name__ == "__main__": 
    main()

Optimal Schedule:
Course: MII226602 - ('Jaringan Komputer Lanjut', 'Rabu', '10:30-13:00')
Course: MII226502 - ('Sistem Temu Balik Informasi', 'Kamis', '07:30-10:00')
Course: MII226205 - ('Pengolahan dan Analisis Citra Digital', 'Kamis', '13:30-16:00')
Course: MII226506 - ('Manajemen dan Audit Sistem Informasi', 'Rabu', '13:30-16:00')
Course: MII226601 - ('Komputasi Awan dan Keamanan Siber', 'Selasa', '13:30-16:00')
Course: MII226501 - ('Data Science', 'Rabu', '10:30-13:00')
